In [ ]:
%%capture
!pip install bertopic datasets accelerate bitsandbytes xformers adjustText

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
data = pd.read_csv('drive/MyDrive/F24 Research/BLMHistory_Tweets.csv', index_col=0)\
    .sample(frac=0.4, random_state=42)
data.head()

,raw_text,created_at,id,account_id,cleaned_text,in_thread,time_diff,chain_id
40864,"Hardly home, but always WORKIN....\n\nOur Incl...",2020-11-03 22:34:46,1.323756e+18,1460836338,hardly home always workin inclusive democracy ...,False,0 days 00:23:13,47432
153165,Tw: police violence and murder \n\nCops in Akr...,2022-07-08 16:47:30,1.545450e+18,1267936704503197699,tw police violence murder cops akron ohio murd...,True,1 days 00:42:32,30603
48909,DACA Advocate On Next Steps After Supreme Cour...,2020-06-25 17:03:02,1.276199e+18,18810464,daca advocate next steps supreme court ruling ...,False,0 days 00:42:58,57151
104171,Sooo the @SacPolice helicopter circled us for ...,2021-10-09 22:18:31,1.446963e+18,4049850912,sooo sacpolice helicopter circled us good minu...,False,0 days 05:00:47,122743
40166,"Shout out to House Democratic Leader, @RepMcCl...",2021-09-15 13:36:34,1.438135e+18,1456075993,shout house democratic leader repmcclinton ple...,False,0 days 13:15:13,46692


In [ ]:
text = data["cleaned_text"].to_list()
del data

In [ ]:
from torch import cuda

# model_id = 'meta-llama/Llama-2-7b-chat-hf'
model_id = 'meta-llama/Llama-2-7b-chat-hf'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

print(device)

cuda:0


In [ ]:
from torch import bfloat16
import transformers

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_quant_type='nf4',  # Normalized float 4
    bnb_4bit_use_double_quant=True,  # Second quantization after the first
    bnb_4bit_compute_dtype=bfloat16  # Computation type
)

In [ ]:
# Llama 2 Tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

# Llama 2 Model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto",
)
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=

In [ ]:
# Our text generator
generator = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    temperature=0.1,
    max_new_tokens=500,
    repetition_penalty=1.1
)

In [ ]:
from sentence_transformers import SentenceTransformer

# Pre-calculate embeddings
embedding_model = SentenceTransformer("BAAI/bge-small-en-v1.5", device=device)
embeddings = embedding_model.encode(text, show_progress_bar=True)

Batches:   0%|          | 0/2598 [00:00<?, ?it/s]

In [ ]:
from umap import UMAP
from hdbscan import HDBSCAN

# umap_model = UMAP(n_neighbors=10, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
# hdbscan_model = HDBSCAN(min_cluster_size=5000, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

umap_model = UMAP(n_neighbors=15, n_components=5,
                  min_dist=0.0, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=int(len(
    text)/20), metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [ ]:
# %%capture
# # Pre-reduce embeddings for visualization purposes
# reduced_embeddings = UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine', random_state=42).fit_transform(embeddings)

In [ ]:
# System prompt describes information given to all conversations
system_prompt = """
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant for labeling topics.
<</SYS>>
"""
# Example prompt demonstrating the output we are looking for
example_prompt = """
I have a topic that contains the following documents:
- Traditional diets in most cultures were primarily plant-based with a little meat on top, but with the rise of industrial style meat production and factory farming, meat has become a staple food.
- Meat, but especially beef, is the word food in terms of emissions.
- Eating meat doesn't make you a bad person, not eating meat doesn't make you a good one.

The topic is described by the following keywords: 'meat, beef, eat, eating, emissions, steak, food, health, processed, chicken'.

Based on the information about the topic above, please create a short label of this topic. Make sure you to only return the label and nothing more.

[/INST] Environmental impacts of eating meat
"""
# Our main prompt with documents ([DOCUMENTS]) and keywords ([KEYWORDS]) tags
main_prompt = """
[INST]
I have a topic that contains the following documents:
[DOCUMENTS]

The topic is described by the following keywords: '[KEYWORDS]'.

Based on the information about the topic above, please create a short label of this topic. Make sure you to only return the label and nothing more.
[/INST]
"""
prompt = system_prompt + example_prompt + main_prompt

In [ ]:
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, TextGeneration
top_n_words = 15
# KeyBERT
keybert = KeyBERTInspired(top_n_words=top_n_words)

# MMR
mmr = MaximalMarginalRelevance(diversity=0.3, top_n_words=top_n_words)

# Text generation with Llama 2
llama2 = TextGeneration(generator, prompt=prompt)

# All representation models
representation_model = {
    "Llama2": llama2,
    "KeyBERT": keybert,
    "MMR": mmr,
}

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(

    # Sub-models
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    representation_model=representation_model,

    # Hyperparameters
    top_n_words=top_n_words,
    verbose=True
)

# Train model
topics, probs = topic_model.fit_transform(text, embeddings)

2024-09-25 07:21:20,322 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-09-25 07:23:46,651 - BERTopic - Dimensionality - Completed ✓
2024-09-25 07:23:46,656 - BERTopic - Cluster - Start clustering the reduced embeddings


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGKILL(-9)}

In [ ]:
topic_model.get_topic_info().columns

In [ ]:
import os
import json
import numpy as np
import time


def convert_and_flatten(data):
    if isinstance(data, dict):
        return {key: convert_and_flatten(value) for key, value in data.items()}
    elif isinstance(data, list):
        flatList = []
        for item in data:
            if isinstance(item, list):
                flatList.extend(item)
            else:
                flatList.append(item)
        return flatList
    elif isinstance(data, np.float32):
        return float(data)
    return data


info = {}

for topic in topic_model.get_topic_info()["Topic"].to_list():
    data = {}
    for key in topic_model.get_topic(topic, full=True).keys():
        data[key] = topic_model.get_topic(topic, full=True)[key]
    data = {key: [[str(item[0]), float(item[1])] for item in value]
            for key, value in data.items()}
    info[topic] = convert_and_flatten(data)

timestr = time.strftime("%Y%m%d-%H%M%S")
with open(f"drive/MyDrive/F24 Research/data/data-{timestr}.json", "w") as f:
    json.dump(info, f, indent=2)